In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


def create_gaussian_kernel(size, sigma=1.0):
    """Creates a 2D Gaussian kernel for smoothing."""

    half_size = size // 2
    x, y = np.mgrid[-half_size:half_size+1, -half_size:half_size+1]
    gaussian_kernel = np.exp(-(x**2 + y**2) / (2 * sigma**2))
    return gaussian_kernel / gaussian_kernel.sum()

def create_log_kernel(size, sigma=1.0):
    """Creates a 2D Laplacian of Gaussian (LoG) kernel for sharpening."""

    half_size = size // 2
    x, y = np.mgrid[-half_size:half_size+1, -half_size:half_size+1]
    term1 = -1 / (np.pi * sigma**4)
    term2 = 1 - ((x**2 + y**2) / (2 * sigma**2))
    term3 = np.exp(-(x**2 + y**2) / (2 * sigma**2))
    log_kernel = term1 * term2 * term3
    return log_kernel - log_kernel.mean()

def apply_convolution(image, kernel):
    """Applies a convolution operation to a single-channel image."""
    return cv2.filter2D(image, -1, kernel)

def create_colored_channel_image(channel, color):
    """
    Creates a BGR image from a single channel, displayed in its specific color.
    Can optionally invert the channel to create a "sketch" effect.
    """
    
    zeros = np.zeros_like(channel)
    if color == 'blue':
        return cv2.merge([channel, zeros, zeros])
    elif color == 'green':
        return cv2.merge([zeros, channel, zeros])
    elif color == 'red':
        return cv2.merge([zeros, zeros, channel])

def display_image_grid(images, titles, grid_shape, figure_title, figsize=(12, 8)):
    """A general function to display a grid of images with titles."""

    rows, cols = grid_shape
    plt.figure(figsize=figsize)
    plt.suptitle(figure_title, fontsize=16)
    for i, (image, title) in enumerate(zip(images, titles)):
        plt.subplot(rows, cols, i + 1)
        if len(image.shape) == 3:
            plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        else: # Grayscale
            plt.imshow(image, cmap='gray')
        plt.title(title)
        plt.axis('off')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

# Main Execution Block

if __name__ == "_main_":
    image_path = 'Lena.jpg'
    image_bgr = cv2.imread(image_path)

    # Kernel Creation
    gaussian_kernel = create_gaussian_kernel(size=5, sigma=1.0)
    log_kernel = create_log_kernel(size=7, sigma=1.4)

    # Part 1: Grayscale Processing ---
    image_gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
    image_smoothed_gray = apply_convolution(image_gray, gaussian_kernel)
    image_sharpened_gray = apply_convolution(image_gray, log_kernel)
        
    display_image_grid(
    images=[image_bgr, image_gray, image_smoothed_gray, image_sharpened_gray],
        titles=['Original Color', 'Grayscale', 'Grayscale Smoothed', 'Grayscale Sharpened (LoG)'],
        grid_shape=(2, 2),
        figure_title='Part 1: Grayscale Convolution',
            figsize=(10, 10)
        )

    #  Part 2: Color Image Processing
    b, g, r = cv2.split(image_bgr)

    # 2a. Gaussian Smoothing on RGB channels
    b_gauss = apply_convolution(b, gaussian_kernel)
    g_gauss = apply_convolution(g, gaussian_kernel)
    r_gauss = apply_convolution(r, gaussian_kernel)
        
    display_image_grid(
        images=[
            create_colored_channel_image(b, 'blue'), create_colored_channel_image(g, 'green'),
            create_colored_channel_image(r, 'red'), image_bgr,
            create_colored_channel_image(b_gauss, 'blue'), create_colored_channel_image(g_gauss, 'green'),
            create_colored_channel_image(r_gauss, 'red'), cv2.merge([b_gauss, g_gauss, r_gauss])
        ],
        titles=[
            'Original Blue', 'Original Green', 'Original Red', 'Original Image',
            'Gaussian Blue', 'Gaussian Green', 'Gaussian Red', 'Merged Result'
        ],
        grid_shape=(2, 4),
            figure_title='Part 2a: Gaussian Smoothing on RGB Channels'
    )

    # 2b. LoG Sharpening on RGB channels
    b_log = apply_convolution(b, log_kernel)
    g_log = apply_convolution(g, log_kernel)
    r_log = apply_convolution(r, log_kernel)
    merged_log = cv2.merge([
        cv2.normalize(b_log, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U),
        cv2.normalize(g_log, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U),
        cv2.normalize(r_log, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    ])

    display_image_grid(
        images=[
            create_colored_channel_image(b_log, 'blue'), create_colored_channel_image(g_log, 'green'),
            create_colored_channel_image(r_log, 'red'), merged_log
        ],
        titles=['LoG on Blue Channel', 'LoG on Green Channel', 'LoG on Red Channel', 'Merged Result'],
            grid_shape=(1, 4),
            figure_title='Part 2b: LoG Sharpening on RGB Channels',
            figsize=(12, 5)
        )

    # 2b. HSV Channel Convolution

    image_hsv = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(image_hsv)

    # Apply Gaussian smoothing
    # Note: Filtering H (Hue) is often avoided, but done here per assignment. The most meaningful result comes from filtering V (Value/Brightness).
    h_gauss_hsv = apply_convolution(h, gaussian_kernel)
    s_gauss_hsv = apply_convolution(s, gaussian_kernel)
    v_gauss_hsv = apply_convolution(v, gaussian_kernel)
    merged_gauss_hsv = cv2.cvtColor(cv2.merge([h_gauss_hsv, s_gauss_hsv, v_gauss_hsv]), cv2.COLOR_HSV2BGR)

    # Apply LoG sharpening
    h_log_hsv = apply_convolution(h, log_kernel)
    s_log_hsv = apply_convolution(s, log_kernel)
    v_log_hsv = apply_convolution(v, log_kernel)
    merged_log_hsv = cv2.cvtColor(cv2.merge([
        cv2.normalize(h_log_hsv, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U),
        cv2.normalize(s_log_hsv, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U),
        cv2.normalize(v_log_hsv, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        ]), cv2.COLOR_HSV2BGR)
        
    display_image_grid(
        images=[image_bgr, merged_gauss_hsv, merged_log_hsv],
        titles=['Original Image', 'Smoothed in HSV Space', 'Sharpened in HSV Space'],
        grid_shape=(1, 3),
        figure_title='Part 2c: Convolution in HSV Space',
        figsize=(12, 6)
    )

    # Filtering V (Value/Brightness).

    # Apply Gaussian smoothing only to V channel
    v_gauss_only = apply_convolution(v, gaussian_kernel)
    merged_gauss_v_only = cv2.cvtColor(cv2.merge([h, s, v_gauss_only]), cv2.COLOR_HSV2BGR)

    # Apply LoG sharpening only to V channel
    v_log_only = apply_convolution(v, log_kernel)
    v_log_only_norm = cv2.normalize(v_log_only, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    merged_log_v_only = cv2.cvtColor(cv2.merge([h, s, v_log_only_norm]), cv2.COLOR_HSV2BGR)

    display_image_grid(
        images=[image_bgr, merged_gauss_v_only, merged_log_v_only],
        titles=['Original Image', 'Gaussian on V Channel', 'LoG on V Channel'],
        grid_shape=(1, 3),
        figure_title='Filtering Only V Channel in HSV',
        figsize=(12, 6)
    )